In [1]:
import pprint
from boardgamegeek import BGGClient
from boardgamegeek import CacheBackendSqlite
import pandas as pd
from pytictoc import TicToc
import pprint

In [2]:
# Create an instance of the API handler:
pathToCache = "/Users/taylorsmith/Python/BGG/cache01.db"
bgg = BGGClient(timeout=15, retries=10, retry_delay=5, disable_ssl=False, requests_per_minute=30,cache=CacheBackendSqlite(path=pathToCache, ttl=3600))

# Test on my uname
u = bgg.user('Brunty_')

## Get the list of the Top 100 Games and the Hot games and combine them into one set

In [6]:
##### Get the set of hot games, show it
hotGames = bgg.hot_items('boardgame')

gameList = set()
cnt = 1
for item in hotGames.items:
    #print(str(cnt)+' '+item.name)
    cnt +=1
#    statusList.append(False) # Initialize all games to not done
    gameList.add(int(item.id))

    
##### Get the Top 100 Games from BGG
import requests
import re

# Get the page for the top 100 rated board games 
r = requests.get('https://boardgamegeek.com/browse/boardgame/page/1?sort=rank&sortdir=asc')
r = requests.get('https://boardgamegeek.com/browse/boardgame/page/2?sort=rank&sortdir=asc')
r = requests.get('https://boardgamegeek.com/browse/boardgame/page/3?sort=rank&sortdir=asc')

# Get the game names- Less reliable than just getting the gameId
#topGames= (re.findall(r'<img alt=\"Board Game: ([^]]*)\"   src=', r.text))

# Get the gameids 
topGames = (re.findall(r'\t\t\[<a href="/boardgame/([^]]*)/marketplace/geekmarket', r.text))

for item in topGames:
    try:
        gid = item.split('/')[0]
        gameList.add(int(gid))
    except:
        pass

    
#for item in gameList:
#    print('Game {} is called {}'.format(item, bgg.game(game_id=item, comments=False,rating_comments=False).name))
    

# Make the list of the hot games to add to the list of ones to process
#progressList = dict(zip(gameList,statusList))

# Check the list of the games already in the dataset so you don't waste time adding them again


In [7]:
# If the file exists get all the values from the , otherwise mark them all False
import os.path
from os import path

path_Dataset = "Datasets/Ratings.csv"
if(path.exists(path_Dataset)):
    print('Dataset found at {} adding to this one...'.format(path_Dataset))
    df = pd.read_csv(path_Dataset,sep='|')
    completedGameIDs = set(df['Game id'])
    
    # Remove the ones that we already did
    gameList = gameList - completedGameIDs
    HeaderOption = False
        
    # Open it and check which ones are done
else:
    print('No dataset started at {} starting a new one...'.format(path_Dataset))
    HeaderOption = True

# Dump the set to a list and give it all statuses
progressList = {x : y for x in list(gameList) for y in [False]}

Dataset found at Datasets/Ratings.csv adding to this one...


In [8]:
len(progressList)

99

In [ ]:
# For a given list of games, get the data and append it to a list, then we'll write it to a data frame
ratingList = []
cnt = 0
for gameid, status in progressList.items():
    ratingListThisGame = []
    try:
        firstTry = True
        while(not(status)): #If we haven't gotten the results here, get them
            try:
                g = bgg.game(game_id=gameid,rating_comments=True,comments=False)

                try:
                    for game_comment in g.comments:
                        #usrinfo = bgg.user(g.comments[0].commenter) # How to get a single user
                        try:
                            ratingListThisGame.append([g.name, int(g.id), game_comment.commenter, int(10*float(game_comment.rating))])
                            ratingList.append([g.name, int(g.id), game_comment.commenter, int(10*float(game_comment.rating))])
                        except:
                            print("\t Error retrieving rating info for user: {}, skipping!".format(game_comment.commenter))
                            pass


                    print('Finished getting the {} ratings for {} successfully'.format(len(g.comments), g.name)) 
                    progressList[gameid] = True
                    status = True
                    break

                except:
                    print("\t Error retrieving rating info for user: {}, skipping!".format(game_comment.commenter))
                    pass
            except:
                if(firstTry):
                    print("\nError retrieving game info for {}, retrying.".format(bgg.game(game_id=gameid).name),end='')
                    firstTry = False
                else:
                    print('.',end='')
                
        else:
            print('Ratings for {} already pulled successfully, continuing'.format(bgg.game(game_id=gameid).name))    
            pass
    
        # Then we'll write this game to a data frame and show some of it
        bgg_thisgame_df = pd.DataFrame(ratingListThisGame,columns=['Game Name','Game id','User Name', 'Rating'])
        bgg_thisgame_df.to_csv(path_or_buf=path_Dataset, sep='|', na_rep='', float_format=None, columns=None, header=HeaderOption, index=True, index_label=None, mode = 'a')
    except:
        print("Error retrieving game info for {}, skipping!".format(bgg.game(game_id=gameid).name))
        progressList[gameid] = False

Error retrieving game info for Cryptid, skipping!

Error retrieving game info for Die Macher, retrying.

# Then we'll write it to a data frame and show some of it
bgg_df = pd.DataFrame(ratingList,columns=['Game Name','Game id','User Name', 'Rating'])
print(bgg_df)

bgg_df.to_csv(path_or_buf=path_Dataset, sep='|', na_rep='', float_format=None, columns=None, header=HeaderOption, index=True, index_label=None, mode = 'a')

In [ ]:
# Need to make a similar database of all the UserNames and associate them with a userID (or at least a uniqueID for the collaborative filtering)

In [ ]:
print("\nhello")